In [ ]:
from notebooks import *
memory.log.level = 'debug'
sg.init(app=None, skip=[sg_load.load_search_recs])

INFO     [18:21:02.718] [45037]  32 api.server_globals/init

DEBUG    [18:21:02.725] [45037]  75 api.server_globals/load: load_search...

•

 api/server_globals/load_search(version=2)


INFO     [18:21:03.048] [45037]  79 api.server_globals/load: load_search (took 0.316s)

...

DEBUG    [18:21:03.395] [45037]  75 api.server_globals/load: load_d_feats...

INFO     [18:21:03.402] [45037] 110 api.server_globals/load_d_feats

INFO     [18:21:03.407] [45037]  79 api.server_globals/load: load_d_feats (took 0.006s)

WARNING  [18:21:03.412] [45037]  73 api.server_globals/load: load_search_recs [skipped]

INFO     [18:21:03.418] [45037]  34 api.server_globals/init: done

# Faster(/smaller) search_recs load from disk

Takeaways
- Forget .pkl
    - ~3x bigger, ~4–8x slower to read (vs. .sqlite/.parquet)
- Forget .gzip
    - ~10x slowdown
    - TODO Revisit compression once we start sending payloads to mobile clients.
- Start with float32 instead of float16
    - Size: only ~1.2x bigger (.sqlite, .parquet) -- I was expecting ~2x
    - Read time: ~1.8x for .sqlite, ~1.2x for .parquet
    - TODO Revisit float16 later when we need ~1.2–2x gains
- Start building on .sqlite, since .parquet is a dead end for mobile
    - Size: same (< 1%)
    - Read time: .sqlite ~2x slower than .parquet
    - Write time: .sqlite ~10x slower than .parquet, but not a relevant bottleneck, and dominated by etl compute anyway
- FIXME Crap, I forgot to account for the ~20s it takes to convert back _to_ search_recs from the serdes df
    - ...

Perf (from final cell)
```
.pkl                   n[35231] d[float16] file[1.7g] w[ 11s] r[7.0s] uss[904m]
.parquet.uncompressed  n[35231] d[float16] file[607m] w[2.7s] r[1.0s] uss[304m]
.parquet.gzip          n[35231] d[float16] file[530m] w[ 49s] r[8.7s] uss[405m]
.sqlite                n[35231] d[float16] file[610m] w[ 38s] r[1.6s] uss[265m]

.pkl                   n[35231] d[float32] file[2.1g] w[ 13s] r[ 10s] uss[1.1g]
.parquet.uncompressed  n[35231] d[float32] file[736m] w[3.4s] r[1.2s] uss[384m]
.parquet.gzip          n[35231] d[float32] file[652m] w[ 58s] r[ 11s] uss[503m]
.sqlite                n[35231] d[float32] file[740m] w[ 37s] r[2.9s] uss[389m]
```

In [ ]:
search_recs = get_search_recs(cache_type=None)

INFO     [18:21:05.677] [45037] 295 api.recs/get_search_recs

INFO     [18:21:05.686] [45037] 344 api.recs/_compute_search_recs: {len(sg.xc_meta): 35233, countries_k: na, com_names_k: ca, num_recs: null}

•

 api/recs/to_paths_sliced(version=0)


•

...

f_p: [

] |   0% (35231) |  0.6s

f_p: [

#########################################################################################

] | 100% (35231) |  0.7s

In [ ]:
# Inspect types + values
(search_recs
    [:1].T
    .pipe(df_assign_first, type=lambda df: df[0].map(lambda x: type(x).__name__))
)

,type,0
dataset,str,xc
species,str,SNGO
species_com_name,str,Snow Goose
species_query,str,SNGO
duration_s,float,10
samples_mb,float,0.422
samples_n,int,221184
basename,str,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)"
species_longhand,str,Snow Goose - SNGO
id,str,"cache/audio/xc/data/SNGO/160423/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)"


In [ ]:
# How big are f_*?
#   - float16 or float32? -- see takeaways below
print(f'shape: {search_recs.shape}')
for dtype in [
    None,        # total[703m] = no_f_*[116m] + f_*[587m] (= f_f[479m] + f_p[108m]) -- List[float]
    np.float64,  # total[639m] = no_f_*[116m] + f_*[523m] (= f_f[426m] + f_p[ 97m]) -- Same as np.float (np default)
    np.float32,  # total[381m] = no_f_*[116m] + f_*[265m] (= f_f[215m] + f_p[ 50m])
    np.float16,  # total[252m] = no_f_*[116m] + f_*[136m] (= f_f[109m] + f_p[ 27m])
]:
    mem = (search_recs
        # [:1000]  # XXX Faster dev
        .pipe(lambda df: df if dtype is None else (df
            .pipe(df_col_map,
                f_f=lambda xs: np.array(xs, dtype=dtype),  # List[float] -> np.ndarray
                f_p=lambda xs: np.array(xs, dtype=dtype),  # List[float] -> np.ndarray
            )
        ))
        .memory_usage(deep=True)
        .sort_values(ascending=False)
    )
    print()
    print(f'dtype: {dtype}')
    print(f'total: {humanize.naturalsize(mem.sum())}')
    display(mem
        .to_frame().rename(columns={0: 'mem'})
        .sort_values('mem', ascending=True)
        .assign(rev_cumsum=lambda df: df.mem.cumsum())
        .sort_values('mem', ascending=False)
        .assign(cumsum=lambda df: df.mem.cumsum())
        .applymap(humanize.naturalsize)
        [:5]
% {time: 75.128s}
   )

shape: (35231, 58)

dtype: None

total: 537.4 MB

,mem,rev_cumsum,cumsum
feat,426.4 MB,537.4 MB,426.4 MB
id,6.5 MB,111.0 MB,432.9 MB
path,6.5 MB,104.5 MB,439.4 MB
basename,5.4 MB,98.0 MB,444.8 MB
remarks,5.3 MB,92.6 MB,450.2 MB


dtype: <class 'numpy.float64'>

...

total: 795.5 MB

,mem,rev_cumsum,cumsum
feat,426.4 MB,795.5 MB,426.4 MB
f_f,215.1 MB,369.0 MB,641.5 MB
f_p,50.3 MB,154.0 MB,691.8 MB
id,6.5 MB,103.7 MB,698.3 MB
path,6.5 MB,97.2 MB,704.8 MB


dtype: <class 'numpy.float16'>

total: 666.5 MB

,mem,rev_cumsum,cumsum
feat,426.4 MB,666.5 MB,426.4 MB
f_f,109.4 MB,240.0 MB,535.8 MB
f_p,27.0 MB,130.7 MB,562.8 MB
id,6.5 MB,103.7 MB,569.3 MB
path,6.5 MB,97.2 MB,575.8 MB


In [ ]:
# Perf (results recorded in next cell)
def measure(path, msg, f):
    # Various contortions to print time + mem delta + file size on one line
    with print_mem_delta(
        collect_before=True, collect_after=False,
        desc=None,
        format='uss[%(uss)9s]',  # uss > rss,vms [https://psutil.readthedocs.io/en/latest/#psutil.Process.memory_full_info]
        print=partial(print, end='')
    ):
        # Retain ret to ensure mem delta is measured before it's collected
        ret = timed_print(f=f, msg='time[%%s] %-55s' % f'{path} {msg}', print=partial(print, end=''))
    print(' file[%8s]' % naturalsize(Path(path).stat().st_size))
    return ret

run = AttrDict(

    # Params
    version=5,
    n=(
        # len(search_recs) // 100
        # len(search_recs) // 10
        len(search_recs)
    ),
    array_dtypes=[
        np.float16,
        np.float32,
    ],

    # File formats
    # pkl=None,  # Slow
    parquet=[
        'uncompressed',  # Fast
        # 'gzip',  # Slow
        # 'snappy',  # Not installed [should be faster than gzip]
    ],
    sqlite=None,  # Fast

)

for array_dtype in run.array_dtypes:
    path = f'/tmp/df-{run.version}-{run.n}-{array_dtype.__name__}'
    df = (search_recs
        # Limit (for faster dev)
        [:run.n]
        # HACK Drop df_cell cols (currently just *_stack)
        #   - TODO Format the ones that matter as str/bytes (e.g. html)
        [lambda df: [c for c in df if not c.endswith('_stack')]]
        # ~10x faster to serdes np.array than list (but only slightly more compact)
        .pipe(df_col_map,
            f_f=lambda xs: np.array(xs, dtype=array_dtype),  # List[float] -> np.ndarray
            f_p=lambda xs: np.array(xs, dtype=array_dtype),  # List[float] -> np.ndarray
        )
    )
    to_serdes = lambda df: (df
        # Convert types for serdes
        .pipe(df_col_map,
            feat=np_save_to_bytes,          # np.array -> bytes (.parquet, .sqlite)
            f_f=np_save_to_bytes,           # np.array -> bytes (.parquet, .sqlite)
            f_p=np_save_to_bytes,           # np.array -> bytes (.parquet, .sqlite)
            background=json.dumps,          # List[str] -> str (.sqlite)
            background_species=json.dumps,  # List[str] -> str (.sqlite)
        )
    )
    from_serdes = lambda df: (df
        # Convert types from serdes
        .pipe(df_col_map,
            feat=np_load_from_bytes,        # np.array <- bytes (.parquet, .sqlite)
            f_f=np_load_from_bytes,         # np.array <- bytes (.parquet, .sqlite)
            f_p=np_load_from_bytes,         # np.array <- bytes (.parquet, .sqlite)
            background=json.loads,          # List[str] <- str (.sqlite)
            background_species=json.loads,  # List[str] <- str (.sqlite)
        )
    )
    df = measure('/dev/null', 'to_serdes',   lambda: to_serdes(df))
    _  = measure('/dev/null', 'from_serdes', lambda: from_serdes(df))
    if 'pkl' in run:
        pkl_path = f'{path}.pkl'
        measure(pkl_path, 'write', lambda: joblib.dump(df, pkl_path))
        measure(pkl_path, 'read',  lambda: joblib.load(pkl_path))
    if 'parquet' in run:
        for compression in run.parquet:
            parquet_path = f'{path}.parquet.{compression}'
            measure(parquet_path, 'write', lambda: df.to_parquet(parquet_path, engine='fastparquet', compression=compression))
            measure(parquet_path, 'read',  lambda: pd.read_parquet(parquet_path, engine='fastparquet'))
    if 'sqlite' in run:
        sqlite_path = f'{path}.sqlite'
        eng = sqla.create_engine(f'sqlite:///{sqlite_path}')
        with eng.begin() as con:
            measure(sqlite_path, 'write', lambda: df.to_sql('df', con=con, if_exists='replace',
                chunksize=1000,  # Else mem unsafe (default is to write all rows in same operation)
            ))
            measure(sqlite_path, 'read',  lambda: pd.read_sql_table('df', con=con))
        eng.dispose()  # Release conn pools
    print()

time[00:13.071] /dev/null to_serdes                                    

uss[ 464.5 MB]

 file[     0 B]

time[00:19.966] /dev/null from_serdes                                  

uss[ 460.1 MB]

...

uss[  62.2 MB]

 file[740.4 MB]

time[00:01.694] /tmp/df-5-35231-float32.sqlite read                    

uss[ 303.3 MB]

 file[740.5 MB]